# Lesson 6: Computer Systems and Theory

Deep dive into computer architecture, operating systems, and theoretical computer science.

## What You'll Learn
- Computer architecture and organization
- Memory hierarchy and management
- Concurrency and parallelism
- Computational complexity theory
- Formal languages and automata
- Advanced system programming concepts

## Computer Architecture

### Von Neumann Architecture

Modern computers follow the Von Neumann architecture:
- **CPU (Central Processing Unit)**
  - Control Unit: Directs operations
  - ALU (Arithmetic Logic Unit): Performs calculations
  - Registers: Ultra-fast temporary storage
- **Memory**: Stores data and instructions
- **I/O Devices**: Input/Output
- **Bus**: Communication pathway

### CPU Execution Cycle (Fetch-Decode-Execute)

In [ ]:
class SimpleCPU:
    """Simplified CPU simulator."""
    
    def __init__(self):
        self.registers = {'A': 0, 'B': 0, 'C': 0}  # General purpose registers
        self.memory = [0] * 256  # 256 bytes of memory
        self.pc = 0  # Program Counter
        self.ir = None  # Instruction Register
        
    def fetch(self, program):
        """Fetch instruction from memory."""
        if self.pc < len(program):
            self.ir = program[self.pc]
            self.pc += 1
            return True
        return False
    
    def decode_execute(self):
        """Decode and execute instruction."""
        if self.ir is None:
            return
        
        op, *args = self.ir.split()
        
        if op == 'LOAD':
            reg, value = args
            self.registers[reg] = int(value)
            print(f"LOAD: Register {reg} = {value}")
            
        elif op == 'ADD':
            reg1, reg2, dest = args
            self.registers[dest] = self.registers[reg1] + self.registers[reg2]
            print(f"ADD: {reg1} + {reg2} = {self.registers[dest]} -> {dest}")
            
        elif op == 'STORE':
            reg, addr = args
            self.memory[int(addr)] = self.registers[reg]
            print(f"STORE: Register {reg} -> Memory[{addr}]")
            
    def run(self, program):
        """Execute program."""
        print("Starting CPU execution...\n")
        while self.fetch(program):
            self.decode_execute()
        print("\nExecution complete!")
        print(f"Final registers: {self.registers}")

# Test program
program = [
    "LOAD A 10",
    "LOAD B 20",
    "ADD A B C",
    "STORE C 0"
]

cpu = SimpleCPU()
cpu.run(program)

## Memory Hierarchy

From fastest to slowest:
1. **Registers** (~1 cycle, few bytes)
2. **L1 Cache** (~4 cycles, 32-64 KB)
3. **L2 Cache** (~10 cycles, 256 KB - 1 MB)
4. **L3 Cache** (~40 cycles, 8-64 MB)
5. **RAM** (~100 cycles, GBs)
6. **SSD/HDD** (millions of cycles, TBs)

### Cache Simulation

In [ ]:
from collections import OrderedDict

class LRUCache:
    """Least Recently Used cache implementation."""
    
    def __init__(self, capacity):
        self.cache = OrderedDict()
        self.capacity = capacity
        self.hits = 0
        self.misses = 0
        
    def get(self, key):
        """Get value from cache."""
        if key in self.cache:
            self.hits += 1
            # Move to end (most recently used)
            self.cache.move_to_end(key)
            return self.cache[key]
        else:
            self.misses += 1
            return None
    
    def put(self, key, value):
        """Put value in cache."""
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            # Remove least recently used
            self.cache.popitem(last=False)
    
    def hit_rate(self):
        """Calculate cache hit rate."""
        total = self.hits + self.misses
        return self.hits / total if total > 0 else 0

# Simulate memory access patterns
cache = LRUCache(capacity=4)
memory_accesses = [1, 2, 3, 4, 1, 2, 5, 1, 2, 3, 4, 5]

print("Memory Access Simulation:")
for addr in memory_accesses:
    value = cache.get(addr)
    if value is None:
        print(f"Address {addr}: MISS - Loading from memory")
        cache.put(addr, f"data_{addr}")
    else:
        print(f"Address {addr}: HIT - Retrieved from cache")

print(f"\nCache Hit Rate: {cache.hit_rate() * 100:.1f}%")
print(f"Hits: {cache.hits}, Misses: {cache.misses}")

## Concurrency and Parallelism

- **Concurrency**: Multiple tasks making progress (interleaved)
- **Parallelism**: Multiple tasks running simultaneously (true parallel execution)

### Multithreading

In [ ]:
import threading
import time

class BankAccount:
    """Thread-safe bank account."""
    
    def __init__(self, balance=0):
        self.balance = balance
        self.lock = threading.Lock()  # Prevent race conditions
        
    def deposit(self, amount):
        with self.lock:  # Acquire lock
            current = self.balance
            time.sleep(0.001)  # Simulate processing
            self.balance = current + amount
            
    def withdraw(self, amount):
        with self.lock:
            current = self.balance
            time.sleep(0.001)
            if current >= amount:
                self.balance = current - amount
                return True
            return False

# Test concurrent transactions
account = BankAccount(1000)

def make_deposits(account, count):
    for _ in range(count):
        account.deposit(10)

def make_withdrawals(account, count):
    for _ in range(count):
        account.withdraw(5)

# Create threads
threads = []
threads.append(threading.Thread(target=make_deposits, args=(account, 100)))
threads.append(threading.Thread(target=make_withdrawals, args=(account, 100)))

# Start all threads
start_time = time.time()
for thread in threads:
    thread.start()

# Wait for completion
for thread in threads:
    thread.join()

end_time = time.time()

print(f"Final balance: ${account.balance}")
print(f"Expected: ${1000 + (100 * 10) - (100 * 5)}")
print(f"Time taken: {end_time - start_time:.2f}s")

### Multiprocessing

In [ ]:
from multiprocessing import Pool, cpu_count
import math

def is_prime(n):
    """Check if number is prime (computationally intensive)."""
    if n < 2:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

# Compare sequential vs parallel execution
numbers = range(10000, 10100)

# Sequential
start = time.time()
sequential_primes = [n for n in numbers if is_prime(n)]
sequential_time = time.time() - start

# Parallel
start = time.time()
with Pool(processes=cpu_count()) as pool:
    results = pool.map(is_prime, numbers)
    parallel_primes = [n for n, is_p in zip(numbers, results) if is_p]
parallel_time = time.time() - start

print(f"Found {len(sequential_primes)} primes")
print(f"\nSequential time: {sequential_time:.4f}s")
print(f"Parallel time: {parallel_time:.4f}s")
print(f"Speedup: {sequential_time / parallel_time:.2f}x")
print(f"Using {cpu_count()} CPU cores")

## Computational Complexity Theory

### P vs NP Problem

- **P**: Problems solvable in polynomial time
- **NP**: Problems verifiable in polynomial time
- **NP-Complete**: Hardest problems in NP
- **NP-Hard**: At least as hard as NP-Complete

### Traveling Salesman Problem (NP-Hard)

In [ ]:
from itertools import permutations
import numpy as np

def tsp_brute_force(distance_matrix):
    """Solve TSP using brute force - O(n!)."""
    n = len(distance_matrix)
    cities = list(range(n))
    min_distance = float('inf')
    best_path = None
    
    # Try all permutations
    for perm in permutations(cities[1:]):
        path = [0] + list(perm) + [0]
        distance = sum(distance_matrix[path[i]][path[i+1]] 
                      for i in range(len(path)-1))
        
        if distance < min_distance:
            min_distance = distance
            best_path = path
    
    return best_path, min_distance

# Small example (5 cities)
distances = np.array([
    [0, 10, 15, 20, 25],
    [10, 0, 35, 25, 30],
    [15, 35, 0, 30, 20],
    [20, 25, 30, 0, 15],
    [25, 30, 20, 15, 0]
])

path, distance = tsp_brute_force(distances)
print(f"Best path: {path}")
print(f"Total distance: {distance}")
print(f"\nNote: For 10 cities, this would try {math.factorial(9):,} permutations!")

### Approximation Algorithm (Greedy)

In [ ]:
def tsp_greedy(distance_matrix):
    """Approximate TSP solution using greedy nearest neighbor."""
    n = len(distance_matrix)
    unvisited = set(range(1, n))
    path = [0]
    total_distance = 0
    
    current = 0
    while unvisited:
        # Find nearest unvisited city
        nearest = min(unvisited, key=lambda city: distance_matrix[current][city])
        total_distance += distance_matrix[current][nearest]
        path.append(nearest)
        unvisited.remove(nearest)
        current = nearest
    
    # Return to start
    total_distance += distance_matrix[current][0]
    path.append(0)
    
    return path, total_distance

greedy_path, greedy_distance = tsp_greedy(distances)
print(f"Greedy path: {greedy_path}")
print(f"Greedy distance: {greedy_distance}")
print(f"Optimal distance: {distance}")
print(f"Approximation ratio: {greedy_distance / distance:.2f}x")

## Formal Languages and Automata

### Finite State Machine

In [ ]:
class FiniteStateMachine:
    """Simple FSM for pattern matching."""
    
    def __init__(self):
        self.state = 'START'
        self.transitions = {
            ('START', '0'): 'S1',
            ('START', '1'): 'START',
            ('S1', '0'): 'S1',
            ('S1', '1'): 'S2',
            ('S2', '0'): 'S3',
            ('S2', '1'): 'START',
            ('S3', '0'): 'S1',
            ('S3', '1'): 'ACCEPT'
        }
        self.accept_states = {'ACCEPT'}
    
    def process(self, input_string):
        """Process input string through FSM."""
        self.state = 'START'
        
        for char in input_string:
            key = (self.state, char)
            if key in self.transitions:
                self.state = self.transitions[key]
                print(f"Input: {char} -> State: {self.state}")
            else:
                print(f"Invalid input: {char}")
                return False
        
        return self.state in self.accept_states

# FSM that accepts strings matching pattern "0101"
fsm = FiniteStateMachine()

test_strings = ["0101", "010", "1010", "00101"]
for test in test_strings:
    print(f"\nTesting: {test}")
    result = fsm.process(test)
    print(f"Accepted: {result}")

## Virtual Memory and Paging

In [ ]:
class PageTable:
    """Simulate virtual memory paging."""
    
    def __init__(self, num_pages, page_size):
        self.page_size = page_size
        self.num_pages = num_pages
        self.page_table = {}  # Virtual page -> Physical frame
        self.physical_memory = [None] * num_pages
        self.next_frame = 0
        self.page_faults = 0
        
    def translate(self, virtual_address):
        """Translate virtual address to physical address."""
        page_num = virtual_address // self.page_size
        offset = virtual_address % self.page_size
        
        if page_num not in self.page_table:
            # Page fault!
            self.page_faults += 1
            self.page_table[page_num] = self.next_frame
            self.physical_memory[self.next_frame] = f"Page_{page_num}"
            self.next_frame = (self.next_frame + 1) % self.num_pages
            print(f"Page fault! Loading page {page_num}")
        
        frame = self.page_table[page_num]
        physical_address = frame * self.page_size + offset
        return physical_address

# Simulate memory accesses
pt = PageTable(num_pages=4, page_size=100)

virtual_addresses = [0, 100, 250, 350, 50, 450]

print("Virtual Memory Access Simulation:\n")
for vaddr in virtual_addresses:
    paddr = pt.translate(vaddr)
    print(f"Virtual: {vaddr:3d} -> Physical: {paddr:3d}")

print(f"\nTotal page faults: {pt.page_faults}")

## Exercise

Advanced systems project:

1. **Build a process scheduler**:
   - Implement Round Robin scheduling
   - Implement Priority scheduling
   - Compare average waiting time and turnaround time

2. **Implement a mutex and semaphore**:
   - Create a thread-safe bounded buffer (producer-consumer problem)
   - Handle multiple producers and consumers
   - Prevent deadlock

3. **Design a memory allocator**:
   - Implement first-fit, best-fit, and worst-fit algorithms
   - Handle memory fragmentation
   - Track allocation efficiency

4. **Create a simple virtual machine**:
   - Design bytecode instruction set
   - Implement stack-based operations
   - Support function calls and returns

In [ ]:
# Your code here

